#### 
Pradhyumn Singh
GTID: 903630476

### Libraries Used:

In [1]:
#Basic data loading/storing/manipulation libraries
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader as pdr
from sklearn.preprocessing import RobustScaler

#Plotting Library
import seaborn as sns

#Libraries for web scrapping S&P data
import bs4 as bs
import requests
import pickle

#Libraries for feature creation & testing
from statsmodels.stats.outliers_influence import variance_inflation_factor
from ta.momentum import RSIIndicator,kama,PercentagePriceOscillator


#Libraries for implementing the models used 
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

### Implemented Functions:

#### Function for fetching the S&P ticker lists (used as large Universe)

In [2]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text[:-1]
        tickers.append(ticker)
    
    df_tickers = pd.DataFrame(tickers)
    df_tickers.to_csv('sp500tickers.csv', header=False, index=False)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
        
    return tickers

#### Functions for Stock Data Loading & Data Wrangling (Filling, Scaling, Binary Return Conversion)

In [3]:
def load_stock_data(ticker):    
    stock_data=pdr.DataReader(ticker,'yahoo',start_date,end_date)['Adj Close'].to_frame().rename(columns={'Adj Close':''}).add_suffix(ticker)
    
    #filling missing values
    stock_data.ffill(axis=0,inplace=True)
    stock_data.bfill(axis=0,inplace=True)  
    
    return stock_data
 
def scale_data(dataframe):    
    #using Robust Scaler to scale the data (efficient in case of existence of outliers)
    scaler=RobustScaler()
    scaled_data = scaler.fit_transform(dataframe.to_numpy())
    scaled_data = pd.DataFrame(scaled_data, columns=dataframe.columns,index=dataframe.index)
    
    return scaled_data

def calculate_binary_returns(ticker_data):
    return_data=ticker_data.pct_change().add_suffix('_Returns')
    #convert returns into binary values (1 as profit and -1 as zero/negative profit)
    return_data.iloc[:,0]=np.where(return_data.iloc[:,0]>0,1,-1)
    
    return return_data

#### Functions for creating the dataset for explanatory variables (Both fundamental & Technical Indicators)

In [4]:
def create_macro_feature_data():
    #Change in Vix
    Vix_data=pdr.DataReader('^VIX','yahoo',start_date,end_date)['Adj Close'].to_frame().rename(columns={'Adj Close':''}).add_suffix('Vix')
    Vix_data=Vix_data.diff().add_suffix('_%Change')

    #S&P Return
    SPX_data=pdr.DataReader('^GSPC','yahoo',start_date,end_date)['Adj Close'].to_frame().rename(columns={'Adj Close':''}).add_suffix('S&P')
    SPX_data=SPX_data.pct_change().add_suffix('_%Change')

    #S&P GSCI Return
    SPGSCI_data=pdr.DataReader('^SPGSCI','yahoo',start_date,end_date)['Adj Close'].to_frame().rename(columns={'Adj Close':''}).add_suffix('S&PGSCI')
    SPGSCI_data=SPGSCI_data.pct_change().add_suffix('_%Change')

    #WTI Oil futures Return
    WTI_data=pdr.DataReader('CL=F','yahoo',start_date,end_date)['Adj Close'].to_frame().rename(columns={'Adj Close':''}).add_suffix('WTI')
    WTI_data=WTI_data.pct_change().add_suffix('_%Change')

    #Change in Fed Fund rate
    Fed_rate_data=pdr.DataReader("DFF",'fred',start_date,end_date)
    Fed_rate_data=Fed_rate_data.diff(1).add_suffix('_Change')

    #Moody's Seasoned AAA Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
    AAA_treasury_spread=pdr.DataReader('AAA10Y','fred',start_date,end_date).rename(columns={'AAA10Y':'AAA_treasury_spread'})
    AAA_treasury_spread=AAA_treasury_spread.diff(1).add_suffix('_Change')

    #10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity 
    Term_spread=pdr.DataReader('T10Y3M','fred',start_date,end_date).rename(columns={'T10Y3M':'Term_spread'})
    Term_spread=Term_spread.diff(1).add_suffix('_Change')

    feature_data=pd.concat([Vix_data,SPX_data,SPGSCI_data,WTI_data,Fed_rate_data,AAA_treasury_spread,Term_spread],axis=1,join='inner').shift(1)
    feature_data.dropna(inplace=True)
    
    feature_data_scaled=scale_data(feature_data)
        
    return(feature_data_scaled)


def technical_indicators(ticker_data):
    # Percentage Price Oscialltor
    ticker_data['MA_12'] = ticker_data.iloc[:,0].rolling(5).mean()
    ticker_data['MA_26'] = ticker_data.iloc[:,0].rolling(26).mean()
    ticker_data['PPO']= ((ticker_data['MA_12'] - ticker_data['MA_26']) / ticker_data['MA_26'])
    
    # Relative strength indicator
    ticker_data['RSI'] = RSIIndicator(ticker_data.iloc[:,0]).rsi()

    #ticker_data['Last_Closed'] = ticker_data.iloc[:,0]
    ticker_data['Return_Average']=ticker_data.iloc[:,0].pct_change().rolling(5).mean()
    
    ticker_data_scaled=scale_data(ticker_data)
    
    return ticker_data_scaled[['PPO','Return_Average','RSI']]


def check_multicollinearity(feature_data):  
    vif_data = pd.DataFrame()
    vif_data["feature"] = feature_data.columns
    vif_data["VIF"] = [variance_inflation_factor(feature_data.values, i) for i in range(len(feature_data.columns))]
    
    vif_data.set_index('feature',inplace=True)
    print(vif_data)

    sns.set(rc = {'figure.figsize':(10,8)})
    sns.heatmap(feature_data.corr(), cbar=False, cmap="BuGn", linewidths=3, annot=True)


#### Functions for Model Predictions & Evaluation Metrics

In [27]:
def KNN (X_train, X_test, y_train, y_test,params,tuning):
    
    if tuning==1:
        knn_model = KNeighborsClassifier(**params)
    else:
        knn_model = KNeighborsClassifier()

    
    knn_model.fit(X_train, y_train)
    
    # predictions
    y_predicted=knn_model.predict(X_test)
        
    #Accuracy Metrics: 
    precision=metrics.precision_score(y_test, y_predicted)
    recall=metrics.recall_score(y_test, y_predicted)
    F1_score=metrics.f1_score(y_test, y_predicted)
    accuracy=metrics.accuracy_score(y_test, y_predicted)
        
    y_pred_prob = knn_model.predict_proba(X_test)[::,1]
    AUC_score=metrics.roc_auc_score(y_test, y_pred_prob)
    
    return accuracy, precision, recall, F1_score, AUC_score


def LR (X_train, X_test, y_train, y_test,params,tuning):
    if tuning==1:
        lr_model = LogisticRegression(**params)
    else:
        lr_model = LogisticRegression()
    
    lr_model.fit(X_train, y_train)
    
    # predictions
    y_predicted=lr_model.predict(X_test)
    
    #Accuracy Metrics: 
    precision=metrics.precision_score(y_test, y_predicted)
    recall=metrics.recall_score(y_test, y_predicted)
    F1_score=metrics.f1_score(y_test, y_predicted)
    accuracy=metrics.accuracy_score(y_test, y_predicted)
    
    y_pred_prob = lr_model.predict_proba(X_test)[::,1]
    AUC_score=metrics.roc_auc_score(y_test, y_pred_prob)
    
    return accuracy, precision, recall, F1_score, AUC_score

#### Functions for Hyperparameter Tuning

In [21]:
# For KNN:

def knn_hyper_parameter_tuning(X_train,y_train):
    param_grid = {
    'n_neighbors': [5,7,9,11,13,15] }

    knn_model = KNeighborsClassifier()
        
    # Grid search of parameters, using 3 fold cross validation, 
    grid_search = GridSearchCV(estimator = knn_model, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)

    # Fitting the grid search model
    grid_search.fit(X_train, y_train)

    return grid_search.best_params_


#For Logistic Regression:

def lr_hyper_parameter_tuning(X_train,y_train):
    param_grid = {
    'penalty': ['elasticnet'],
    'solver': ['saga'],
    'l1_ratio': np.linspace(0,1,11),
    'max_iter':[500]
    }
    
        
    lr_model = LogisticRegression()
            
    # Grid search of parameters, using 3 fold cross validation, 
    grid_search = GridSearchCV(estimator = lr_model, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

    # Fitting the grid search model
    grid_search.fit(X_train, y_train)
    
    return grid_search.best_params_

#### Main Function to generate the output dataframe

In [7]:
def generate_output (ticker_list,feature_data_scaled,tuning):
    output_frame=pd.DataFrame(columns=['Ticker','Accuracy_KNN_Model','Precision__KNN_Model','Recall_KNN_Model','F1_Score_KNN_Model','AUC_KNN_Model','Accuracy_LR_Model','Precision_LR_Model','Recall_LR_Model','F1_Score_LR_Model','AUC_LR_Model'])

    for ticker in ticker_list:
        
        #Error handling for those tickers in the large sample for which information is not available
        try: 
            stock_data=load_stock_data(ticker)
        except:
            continue
        
        return_data=calculate_binary_returns(stock_data)
        technical_data=technical_indicators(stock_data).shift(1)
    
        #combined_data=pd.concat([return_data,feature_data_scaled,technical_data],join='inner',axis=1)
        combined_data=pd.concat([return_data,feature_data_scaled,technical_data],join='inner',axis=1)
        combined_data.dropna(inplace=True)
        
        
        #Only running the model for the stocks for which data of atleast two months is available 
        if combined_data.shape[0]<60:
            continue
        
        #Splitting the dataset into explanatory and explained variable
        X=combined_data.iloc[:,1:]
        y=combined_data.iloc[:,0]

        # Splitting dataset into training set and test set
        X_train= X.iloc[0:int(X.shape[0]*60/100),:]
        y_train= y.iloc[0:int(y.shape[0]*60/100)]
        X_test= X.iloc[-(X.shape[0]-X_train.shape[0]):,:]
        y_test= y.iloc[-(y.shape[0]-y_train.shape[0]):,]

        if tuning==1:            
            #finding best parameters for Random Forest
            best_params_knn=knn_hyper_parameter_tuning(X_train, y_train)

            #finding best parameters for Logistic Regression
            best_params_en=lr_hyper_parameter_tuning(X_train, y_train)

            #calculating the evaluation metrics for KNN
            accuracy_m1, precision_m1, recall_m1, F1_score_m1, AUC_score_m1 = KNN(X_train, X_test, y_train, y_test,best_params_knn,tuning)
            
            #calculating the evaluation metrics for Logistic Regression
            accuracy_m2, precision_m2, recall_m2, F1_score_m2, AUC_score_m2 = LR(X_train, X_test, y_train, y_test,best_params_en,tuning)


        else:
            #calculating the evaluation metrics for KNN
            accuracy_m1, precision_m1, recall_m1, F1_score_m1, AUC_score_m1 = KNN(X_train, X_test, y_train, y_test,0,tuning)
            
            #calculating the evaluation metrics for Logistic Regression
            accuracy_m2, precision_m2, recall_m2, F1_score_m2, AUC_score_m2 = LR(X_train, X_test, y_train, y_test,0,tuning)

        output_frame.loc[len(output_frame.index)] = [ticker,accuracy_m1,precision_m1,recall_m1,F1_score_m1,AUC_score_m1,accuracy_m2,precision_m2,recall_m2,F1_score_m2,AUC_score_m2]
      
    return output_frame


### Implementation:

In [28]:
#problem parameters
start_date="2000-01-01"
end_date="2021-11-12"

#Creating the Macro Features and checking for multicollinearity
feature_data_scaled=create_macro_feature_data()
#check_multicollinearity(feature_data_scaled)

#### Output for small sample

In [25]:
#reading the tickers from the csv file
ticker_data_10=pd.read_csv("tickers.csv")
ticker_list_10=ticker_data_10.iloc[:,0].to_list()
hyperparameter_tuning_on=1

output_small_universe=generate_output(ticker_list_10,feature_data_scaled,hyperparameter_tuning_on)

#exporting the data to a csv file
output_small_universe.to_csv('Output_Small_Universe.csv')

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 3 folds for each of 11 candidates, totalling 33 fits
Fitting 5 folds for each of 6 ca

#### Output for large sample

In [12]:
ticker_list_SnP=save_sp500_tickers()

#hyperparameter_tuning kept on off for default to reduce time complexity 
hyperparameter_tuning_on=0
output_large_universe=generate_output(ticker_list_SnP,feature_data_scaled,hyperparameter_tuning_on)

#exporting the data to a csv file
output_large_universe=output_large_universe.loc[(output_large_universe['Precision__KNN_Model']>.5141)& (output_large_universe['Precision_LR_Model']>.5141)]

output_large_universe.sort_values(by='Accuracy_KNN_Model',inplace=True,ascending=False)
output_large_universe_KNN20=output_large_universe.iloc[:20,]
output_large_universe_KNN20.to_csv('Output_Large_Universe_KNN20.csv')

output_large_universe.sort_values(by='Accuracy_LR_Model',inplace=True,ascending=False)
output_large_universe_LR20=output_large_universe.iloc[:20,]
output_large_universe_LR20.to_csv('Output_Large_Universe_LR20.csv')